# Lab 12: Robust Optimization 
In this lab, we will see some applications of robust optimization, namely a modified version of the Knapsack 0/1 problem, and the portfolio optimization problem.

Your job in this lab is to implement the missing functions, and study how different functions lead to different outcomes from both the point of view of the objective value and the probability of violating the constraints of the problem.

The examples are taken from https://xiongpengnus.github.io/rsome/ro_rsome, using the RSOME library for robust optimization.

In [3]:
!pip install rsome > /dev/null
!pip install gurobipy > /dev/null

from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt


# The Knapsack 0/1 Problem
In this exercise, we will solve the Knapsack problem (seen in the previous labs), slightly modified in order to have uncertainties about the volumes of the items.

The uncertainty about the volumes is not the same for all the items. They are defined by $\delta$, defined as a fraction of the size of the volumes of the items.

In this exercise, you are asked to implement the definition of the uncertainty set in order to have both an **ellipsoidal** uncertainty set and a **finite** uncertainty set.

Try to implement different sizes for the ellipsoid and different interval for the finite set and compare the objective values and the probability of violating the constraints with the different setups.

In [74]:
items = [
    {'name': 'apple', 'value': 1, 'volume': 2},    
    {'name': 'pear', 'value': 2, 'volume': 2},    
    {'name': 'banana', 'value': 2, 'volume': 2},    
    {'name': 'watermelon', 'value': 5, 'volume': 10},    
    {'name': 'orange', 'value': 3, 'volume': 2},    
    {'name': 'avocado', 'value': 3, 'volume': 2},    
    {'name': 'blueberry', 'value': 3, 'volume': 1},    
    {'name': 'coconut', 'value': 4, 'volume': 3},    
    {'name': 'cherry', 'value': 2, 'volume': 1},    
    {'name': 'apricot', 'value': 1, 'volume': 1},    
]

N = len(items)
b = 10

c = np.array([i['value'] for i in items]).flatten()  # profit coefficients
w = np.array([i['volume'] for i in items]).flatten()  # profit coefficientss

delta = 0.2*w  # maximum deviations

def robust(get_uncertainty_set):
    """
    The function robust implements the robust optimiztion model,
    given the budget of uncertainty r
    """

    model = ro.Model('robust')
    x = model.dvar(N, vtype='B')  # Boolean variable x (0: leave, 1: keep)
    z = model.rvar(N)             # Random variable 

    # Uncertainty set
    z_set = get_uncertainty_set(z)
    model.max(c @ x)              # Maximize the value of the knapsack (i.e., the dot product between the values and x)

    # Add constraint: the maximum (uncertain) volume is smaller than the budget
    model.st(((w + z*delta) @ x <= b).forall(z_set))

    # Solve
    model.solve(grb, display=False)

    return model.get(), x.get()    # Return the optimal objective and solution


def sim(x_sol, zs):
    """
    The function sim is for calculating the probability of violation
    via simulations.
        x_sol: solution of the Knapsack problem
        zs: random sample of the random variable z
    """

    ws = w + zs*delta   # random samples of uncertain weights

    return (ws @ x_sol > b).mean()

In [256]:
def get_uncertainty_set(z):
    """
    Define an uncertainty set. See the following sources:
    - Ellipsoidal: https://xiongpengnus.github.io/rsome/ro_rsome#section2.2
    - Finite Uncertainty Set
    """
    z_set = (
        # ...                   # Implement this part, try different possibilities
    ) 
    return z_set

In [257]:
num_samples = 20000
zs = np.random.uniform(-1, 1, (num_samples, N))  # Generate random samples for z

objective_value, solution = robust(get_uncertainty_set)
prob_violation = sim(solution, zs)

print(f'Content of the knapsack:')
for i, value in enumerate(solution):
    if value:
        print(f'\t{items[i]["name"]}')
print(f'Total value: {objective_value}. Probability of violation: {prob_violation}')

Content of the knapsack:
	apple
Total value: 3.0. Probability of violation: 0.0


# Robust Portfolio Optimization
In this problem, we want to build a portfolio (e.g., of stocks), by using robust approaches.

To be more specific, in this problem we have a set of fictionary stocks, each of which has different means and deviations for the returns.

Your job here is to implement a **box** uncertainty set to robustly optimize the portfolio.
Try different values for the box in order to study how the uncertainty affects the objective value of and the number of different stocks chosen.

In [270]:
import pprint
n = 10                                 # number of stocks

stocks = {
    f'Company {chr(65+i)}': {'Mean': np.around(np.random.uniform(0.9, 1.1), 2), 'Deviation': np.around(np.random.uniform(0.1, 0.3), 2)}
    for i in range(n)
}

print('Stocks available')
pprint.pprint(stocks)


def portfolio_optimization(get_uncertainty_set):
    p = np.array([stocks[s]['Mean'] for s in stocks])  # mean returns
    delta = np.array([stocks[s]['Deviation'] for s in stocks])  # deviations of returns
    Gamma = 5                               # budget of uncertainty
    
    model = ro.Model()              
    x = model.dvar(n)                       # fractions of investment
    z = model.rvar(n)                       # random variables

    z_set = get_uncertainty_set(z)
    
    model.maxmin(
        (p + delta*z) @ x,                  # the max-min objective
        z_set
    )
    
    model.st(sum(x) == 1)                   # summation of x is one
    model.st(x >= 0)                        # x is non-negative
    
    model.solve(grb)                        # solve the model by Gurobi
    return model.get(), x.get()

Stocks available
{'Company A': {'Deviation': 0.25, 'Mean': 0.94},
 'Company B': {'Deviation': 0.14, 'Mean': 1.01},
 'Company C': {'Deviation': 0.16, 'Mean': 0.91},
 'Company D': {'Deviation': 0.25, 'Mean': 1.04},
 'Company E': {'Deviation': 0.13, 'Mean': 1.08},
 'Company F': {'Deviation': 0.18, 'Mean': 1.08},
 'Company G': {'Deviation': 0.27, 'Mean': 1.03},
 'Company H': {'Deviation': 0.18, 'Mean': 0.99},
 'Company I': {'Deviation': 0.23, 'Mean': 0.95},
 'Company J': {'Deviation': 0.19, 'Mean': 1.02}}


In [273]:
def get_uncertainty_set_po(z):
    """ 
    Return a box uncertainty set 
    (see https://xiongpengnus.github.io/rsome/example_ro_inv).

    Try different values for the size of the box.
    """
    z_set = (
    )
    return z_set

In [ ]:
import matplotlib.pyplot as plt

obj_val, x_sol = portfolio_optimization(get_uncertainty_set_po)

plt.bar(
    [s.split(' ')[1] for s in stocks],  # Stock names
    x_sol,                              # Fraction of the portfolio 
)
plt.xlabel('Stocks')
plt.ylabel('Fraction of investment')
plt.show()
print('Objective value: {0:0.4f}'.format(obj_val))